In [7]:
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim

from skimage import io, color
from skimage.measure import compare_ssim
from skimage.measure import compare_psnr
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [8]:
gamma = 0.5
sigmaU = 0.5
sigmaW = 10
epochs = 100

model_name = 'SDN_Color_Block3_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
save_dir = os.path.join('Models', model_name)

In [9]:
class Block3(nn.Module):
    def __init__(self, ch, kernel_size=3):
        super(Block3, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, kernel_size, padding=1),
            nn.BatchNorm2d(ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        
        c1 = self.conv1(x)
        c2 = self.conv2(c1+x)
        c3 = self.conv3(c2+x)
        
        return c3

class SDNCNN(nn.Module):
    
    def __init__(self, filters=64, image_channels=3, use_bnorm=True, kernel_size=3):
        super(SDNCNN, self).__init__()
        kernel_size = 3
        padding = 1
        self.conv0 = nn.Conv2d(in_channels=image_channels, out_channels=filters, kernel_size=kernel_size, padding=padding)
        
        self.convOut1 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        self.convOut2 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        self.convOut3 = nn.Conv2d(in_channels=filters, out_channels=image_channels, kernel_size=kernel_size, padding=padding, bias=True)
        
        self.ResBlock1 = Block3(filters)
        self.ResBlock2 = Block3(filters)
        self.ResBlock3 = Block3(filters)
        self.ResBlock4 = Block3(filters)
        self.ResBlock5 = Block3(filters)
        self.ResBlock6 = Block3(filters)
        self.ResBlock7 = Block3(filters)
        
        self._initialize_weights()

    def forward(self, x):

        c0 = self.conv0(x)
        
        c1 = self.ResBlock1(c0)
        
        c2 = self.ResBlock2(c1)
        
        c3 = self.ResBlock3(c2)
        
        c4 = self.ResBlock4(c3)

        c5 = self.ResBlock5(c4+c3)
        c55 = self.convOut1(c5)
        
        c6 = self.ResBlock6(c5+c2)
        c66 = self.convOut2(c6)
        
        c7 = self.ResBlock7(c6+c1)
        c77 = self.convOut3(c7)
        
        noise = c55 + c66 + c77
        rec = x - noise
        
        return rec, noise

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
                
def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir, 'model_*.pth'))
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).pth.*", file_)
            epochs_exist.append(int(result[0]))
        initial_epoch = max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch

def show(x, title=None, cbar=False, figsize=None):
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(x, interpolation='nearest')
    if title:
        plt.title(title)
    if cbar:
        plt.colorbar()
    plt.show()

In [10]:
model = SDNCNN()
initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
if initial_epoch > 0:
    print('resuming by loading epoch %03d' % initial_epoch)
    model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % initial_epoch))

resuming by loading epoch 028


In [11]:
params = list(model.parameters())
k = 0
for i in params:
    l = 1
#     print("该层的结构：" + str(list(i.size())))
    for j in i.size():
        l *= j
#     print("该层参数和：" + str(l))
    k = k + l
print("总参数数量和：" + str(k))

总参数数量和：1563337


## 测试彩色CBSD68

In [7]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'CBSD68')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/CBSD68/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(1, initial_epoch+1):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):

            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            
            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.2f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Block1: Epoch:83, PSNR: 33.995, SSIM: 0.9645

Block2: Epoch:75, PSNR: 33.858, SSIM: 0.9634

Block3: Epoch:14, PSNR: 34.327, SSIM: 0.9671

noAdd:  Epoch:5, PSNR: 28.073, SSIM: 0.8505

## 测试Kodak数据集

In [12]:
with torch.no_grad():
    np.random.seed(42)
    torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'kodak')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_data = './Test_datasets/kodak/*.png'
    test_dir = glob.glob(test_data)
    
    initial_epoch = findLastCheckpoint(save_dir=save_dir)
    for e in range(1, initial_epoch+1):

        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):
            start_time = time.time()

            batch_x = io.imread(test_dir[i])
            u = np.random.normal(0, sigmaU, batch_x.shape)
            w = np.random.normal(0, sigmaW, batch_x.shape)
            noise = np.power(batch_x, gamma)*u + w
            batch_y = batch_x + noise
            
            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)
            
            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0

            psnr_list += [compare_psnr(out, batch_x)]
            
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch:%d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch:1, PSNR: 34.640, SSIM: 0.9614
Epoch:2, PSNR: 34.809, SSIM: 0.9635
Epoch:3, PSNR: 34.800, SSIM: 0.9634
Epoch:4, PSNR: 34.806, SSIM: 0.9635
Epoch:5, PSNR: 34.800, SSIM: 0.9634
Epoch:6, PSNR: 34.799, SSIM: 0.9633
Epoch:7, PSNR: 34.873, SSIM: 0.9641
Epoch:8, PSNR: 34.957, SSIM: 0.9649
Epoch:9, PSNR: 34.956, SSIM: 0.9648
Epoch:10, PSNR: 34.946, SSIM: 0.9646
Epoch:11, PSNR: 34.945, SSIM: 0.9645
Epoch:12, PSNR: 34.955, SSIM: 0.9647
Epoch:13, PSNR: 34.952, SSIM: 0.9649
Epoch:14, PSNR: 34.951, SSIM: 0.9647
Epoch:15, PSNR: 34.952, SSIM: 0.9647
Epoch:16, PSNR: 34.968, SSIM: 0.9650
Epoch:17, PSNR: 34.970, SSIM: 0.9650
Epoch:18, PSNR: 34.967, SSIM: 0.9650
Epoch:19, PSNR: 34.976, SSIM: 0.9651
Epoch:20, PSNR: 34.965, SSIM: 0.9649
Epoch:21, PSNR: 34.971, SSIM: 0.9650
Epoch:22, PSNR: 34.969, SSIM: 0.9649
Epoch:23, PSNR: 34.958, SSIM: 0.9648
Epoch:24, PSNR: 34.971, SSIM: 0.9650
Epoch:25, PSNR: 34.955, SSIM: 0.9647
Epoch:26, PSNR: 34.946, SSIM: 0.9645
Epoch:27, PSNR: 34.960, SSIM: 0.9648
Epoch:28, 

Block1: Epoch:72, PSNR: 34.735, SSIM: 0.9641

Block2: Epoch:79, PSNR: 34.579, SSIM: 0.9625

Block3: Epoch:98, PSNR: 35.126, SSIM: 0.9669

## 测试Poly数据集

In [14]:
with torch.no_grad():
    
    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'poly')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    test_dir = glob.glob('./Test_datasets/poly/real/*.JPG')

    initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
    
    for e in range(1, initial_epoch+1):
        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(test_dir)):
            mean_name = test_dir[i].split('/')[4].split('.')[0][:-4] + 'mean.JPG'
    #             print(mean_name)
            mean_path = './Test_datasets/poly/mean/%s' % mean_name

            batch_x = io.imread(mean_path)
            batch_y = io.imread(test_dir[i])

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) /255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0

            psnr_list += [compare_psnr(out, batch_x)]
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()

        print('Epoch: %d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch: 1, PSNR: 37.756, SSIM: 0.9828
Epoch: 2, PSNR: 37.838, SSIM: 0.9832
Epoch: 3, PSNR: 37.840, SSIM: 0.9833
Epoch: 4, PSNR: 37.819, SSIM: 0.9831
Epoch: 5, PSNR: 37.854, SSIM: 0.9834
Epoch: 6, PSNR: 38.128, SSIM: 0.9841
Epoch: 7, PSNR: 38.155, SSIM: 0.9844
Epoch: 8, PSNR: 38.155, SSIM: 0.9844
Epoch: 9, PSNR: 38.154, SSIM: 0.9844
Epoch: 10, PSNR: 38.150, SSIM: 0.9844
Epoch: 11, PSNR: 38.158, SSIM: 0.9844
Epoch: 12, PSNR: 37.969, SSIM: 0.9839
Epoch: 13, PSNR: 38.089, SSIM: 0.9841
Epoch: 14, PSNR: 38.105, SSIM: 0.9840
Epoch: 15, PSNR: 37.848, SSIM: 0.9839
Epoch: 16, PSNR: 38.145, SSIM: 0.9842


KeyboardInterrupt: 

Block1: Epoch: 73, PSNR: 37.579, SSIM: 0.9822

Block2: Epoch: 76, PSNR: 37.811, SSIM: 0.9836

Block3: Epoch: 93, PSNR: 38.199, SSIM: 0.9846

## 测试Nam数据集

In [18]:
with torch.no_grad():

    output_dir = 'Results_Log/%s'%(model_name)
    output_file_name = 'Results_Log/%s/Log_output_%s.txt'%(model_name, 'Nam')
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    test_dir = './Test_datasets/Real_ccnoise_denoised_part'
    clean_dir = glob.glob(os.path.join(test_dir, '*mean.png'))
    
    result_dir = './Results/Nam15_gamma_%.1f_sigmaU_%.1f_sigmaW_%d'%(gamma, sigmaU, sigmaW)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)

    initial_epoch = findLastCheckpoint(save_dir=save_dir)  # load the last model in matconvnet style
    for e in range(10, 60):
        model = torch.load(os.path.join(save_dir, 'model_%03d.pth' % e))
        model = model.cuda()
        model.eval()

        psnr_list = []
        ssim_list = []

        for i in range(len(clean_dir)):
            batch_x = io.imread(clean_dir[i])
            noise_name = clean_dir[i].split('/')[3].split('.')[0][:-4] + 'real.png'
            noise_path = os.path.join(test_dir, noise_name)
            batch_y = io.imread(noise_path)

            batch_x = torch.from_numpy(batch_x.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()
            batch_y = torch.from_numpy(batch_y.transpose(2,0,1).astype('float32'))[None,:,:,:].cuda()

            out, noise_out = model(batch_y)

            batch_x = batch_x.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0
            out = out.clamp(0, 255)[0,...].cpu().detach().numpy().transpose(1,2,0) / 255.0

            psnr_list += [compare_psnr(out, batch_x)]
            ssim_list += [compare_ssim(out, batch_x, multichannel=True)]
            
#             out_name = noise_name[:-8] + 'PSNR_' + '%.2f'%psnr_list[-1] + '_SSIM_' + '%.4f'%ssim_list[-1] + '.png'
#             io.imsave(os.path.join(result_dir, out_name), out)
#             print('%s----%.2f----%.4f'%(noise_name, np.array(psnr_list[-1]), np.array(ssim_list[-1])))

        output_data = 'Epoch: %d, PSNR: %.2f, SSIM: %.4f\n' % (e, np.mean(psnr_list), np.mean(ssim_list))
        output_file = open(output_file_name, 'a')
        output_file.write(output_data)
        output_file.close()
            
            
        print('Epoch: %d, PSNR: %.3f, SSIM: %.4f' % (e, np.mean(psnr_list), np.mean(ssim_list)))

Epoch: 10, PSNR: 36.284, SSIM: 0.9706
Epoch: 11, PSNR: 36.271, SSIM: 0.9706
Epoch: 12, PSNR: 36.302, SSIM: 0.9709
Epoch: 13, PSNR: 36.242, SSIM: 0.9703
Epoch: 14, PSNR: 36.294, SSIM: 0.9708
Epoch: 15, PSNR: 36.271, SSIM: 0.9705
Epoch: 16, PSNR: 35.396, SSIM: 0.9661
Epoch: 17, PSNR: 35.981, SSIM: 0.9685
Epoch: 18, PSNR: 36.170, SSIM: 0.9696
Epoch: 19, PSNR: 36.151, SSIM: 0.9695
Epoch: 20, PSNR: 36.236, SSIM: 0.9703
Epoch: 21, PSNR: 36.248, SSIM: 0.9703
Epoch: 22, PSNR: 36.085, SSIM: 0.9690
Epoch: 23, PSNR: 36.130, SSIM: 0.9694
Epoch: 24, PSNR: 36.680, SSIM: 0.9726
Epoch: 25, PSNR: 36.667, SSIM: 0.9726
Epoch: 26, PSNR: 36.675, SSIM: 0.9727
Epoch: 27, PSNR: 36.663, SSIM: 0.9728
Epoch: 28, PSNR: 36.643, SSIM: 0.9726
Epoch: 29, PSNR: 36.640, SSIM: 0.9725
Epoch: 30, PSNR: 36.637, SSIM: 0.9725
Epoch: 31, PSNR: 36.662, SSIM: 0.9727
Epoch: 32, PSNR: 36.666, SSIM: 0.9726
Epoch: 33, PSNR: 36.652, SSIM: 0.9727
Epoch: 34, PSNR: 36.659, SSIM: 0.9727
Epoch: 35, PSNR: 36.667, SSIM: 0.9728
Epoch: 36, P

Block1: Epoch: 79, PSNR: 36.572, SSIM: 0.9732

Block2: Epoch: 70, PSNR: 36.735, SSIM: 0.9726

Block3: Epoch: 391, PSNR: 37.509, SSIM: 0.9778

In [19]:
noise_name[:-8] + 'PSNR_' + '%.2f'%psnr_list[-1] + '_SSIM_' + '%.4f'%ssim_list[-1] + '.png'

'5dmark3_iso3200_2_PSNR_37.25_SSIM_0.9803.png'